# Preprocessing

## Setup

In [1]:
import os
import random

import boto3

import numpy as np
import pyspark

import matplotlib.pyplot as plt

import nltk
from nltk.stem.snowball import SnowballStemmer

from pyspark.sql.functions import lit, udf
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import CountVectorizer

from utils import process_review, build_freqs

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [2]:
# set up a giant single executor with many threads and specify memory cap
spark = pyspark.sql.SparkSession \
        .builder \
        .config("spark.executor.instances", 10) \
        .config("spark.driver.memory", "32g") \
        .getOrCreate()
spark.conf.set("spark.sql.analyzer.failAmbiguousSelfJoin", "false")

BUCKET_NAME = 'amazon-reviews-pds'
LOCAL_DIR = os.path.expanduser('~') + '/SageMaker/data/'
s3_client = boto3.client('s3')
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(BUCKET_NAME)

first = True

for object_summary in my_bucket.objects.filter(Prefix="parquet/product_category=Electronics/"):
    OBJECT_NAME = object_summary.key
    LOCAL_FILE = LOCAL_DIR + OBJECT_NAME.split("/")[-1]
    s3_client.download_file(BUCKET_NAME, OBJECT_NAME, LOCAL_FILE)
    data = spark.read.parquet(LOCAL_FILE)
    print(data.count())
    if first:
        first = False
    else:
        data = prev_data.union(data)
    prev_data = data

print(data.count())

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [3]:
data.columns

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [4]:
data=data.select('review_body', 'star_rating')

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [5]:
data = data.filter(data['review_body'].isNotNull())

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [6]:
# Positive reviews have a rating greater than 3, and
# negative reviews have a rating less than 3
positive_reviews = data.where(data['star_rating'] > 3)
# assign sentiment value of 1
positive_reviews=positive_reviews.withColumn("sentiment", lit(1))
NUM_POSITIVE_REVIEWS = positive_reviews.count()
print("Number of positive reviews: ", NUM_POSITIVE_REVIEWS)
negative_reviews = data.where(data['star_rating'] < 3)
# assign sentiment value of 0
negative_reviews=negative_reviews.withColumn("sentiment", lit(0))
NUM_NEGATIVE_REVIEWS = negative_reviews.count()
print("Number of negative reviews: ", NUM_NEGATIVE_REVIEWS)

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [7]:
# free up data
data=0

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [8]:
reviews = positive_reviews.union(negative_reviews)

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [9]:
from pyspark.sql.functions import monotonically_increasing_id 
reviews = reviews.select("*").withColumn("id", monotonically_increasing_id())

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [10]:
positive_reviews=0
negative_reviews=0

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [11]:
# Declare a figure with a custom size
fig = plt.figure(figsize=(5, 5))

# labels for the two classes
labels = 'Positives', 'Negative'

# Sizes for each slide
sizes = [NUM_POSITIVE_REVIEWS, NUM_NEGATIVE_REVIEWS] 

# Declare pie chart, where the slices will be ordered and plotted counter-clockwise:
plt.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)

# Equal aspect ratio ensures that pie is drawn as a circle.
plt.axis('equal')  

# Display the chart
plt.show()

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


from pyspark.sql.functions import rand
[t[0] for t in positive_reviews.select('review_body').orderBy(rand()).limit(3).collect()]

[t[0] for t in negative_reviews.select('review_body').orderBy(rand()).limit(3).collect()]

## Preprocess raw text for sentiment analysis


In [12]:
tokenizer = Tokenizer(outputCol="words")
tokenizer.setInputCol("review_body")

Tokenizer_5e520cd27190

In [13]:
reviews = tokenizer.transform(reviews)

In [14]:
reviews.show(3)

+--------------------+-----------+---------+---+--------------------+
|         review_body|star_rating|sentiment| id|               words|
+--------------------+-----------+---------+---+--------------------+
|These headphones ...|          4|        1|  0|[these, headphone...|
|Arrived as expect...|          5|        1|  1|[arrived, as, exp...|
|the reason I chos...|          4|        1|  2|[the, reason, i, ...|
+--------------------+-----------+---------+---+--------------------+
only showing top 3 rows



In [15]:
remover = StopWordsRemover(inputCol="words", outputCol="tokens")
reviews = remover.transform(reviews)
reviews.show(3)

+--------------------+-----------+---------+---+--------------------+--------------------+
|         review_body|star_rating|sentiment| id|               words|              tokens|
+--------------------+-----------+---------+---+--------------------+--------------------+
|These headphones ...|          4|        1|  0|[these, headphone...|[headphones, nice...|
|Arrived as expect...|          5|        1|  1|[arrived, as, exp...|[arrived, expecte...|
|the reason I chos...|          4|        1|  2|[the, reason, i, ...|[reason, chose, p...|
+--------------------+-----------+---------+---+--------------------+--------------------+
only showing top 3 rows



In [16]:
stemmer = SnowballStemmer(language='english')
stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))
reviews = reviews.withColumn("stemmed_tokens", stemmer_udf("tokens")).select('id', 'stemmed_tokens','sentiment')


In [17]:
reviews.show(3)

+---+--------------------+---------+
| id|      stemmed_tokens|sentiment|
+---+--------------------+---------+
|  0|[headphon, nice.,...|        1|
|  1|[arriv, expect, g...|        1|
|  2|[reason, chose, p...|        1|
+---+--------------------+---------+
only showing top 3 rows

